<a href="https://colab.research.google.com/github/Crossme0809/frenzyTechAI/blob/main/fine-tune-code-llama/fine_tune_code_llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code Llama 微调指南

**在本指南中，我将向您展示如何微调 Code Llama，使其成为 SQL 开发人员的野兽。对于编码任务，您通常可以从 Code Llama 获得比 Llama 2 更好的性能，特别是当您将模型专门用于特定任务时：**

- 使用 [b-mc2/sql-create-context](https://huggingface.co/datasets/b-mc2/sql-create-context) 这是一堆文本查询及其相应的 SQL 查询
- Lora 方法，将基本模型量化为 int 8，冻结其权重，仅训练适配器
- 大部分代码是从 [alpaca-lora](https://github.com/tloen/alpaca-lora) 参考的，同时也进行了一定的改进与优化


### 2. Pip installs


In [14]:
!pip install git+https://github.com/huggingface/transformers.git@main bitsandbytes accelerate  # we need latest transformers for this
!pip install git+https://github.com/huggingface/peft.git@e536616888d51b453ed354a6f1e243fecb02ea08
!pip install datasets==2.10.1
import locale # colab workaround
locale.getpreferredencoding = lambda: "UTF-8" # colab workaround
!pip install wandb

Looking in indexes: https://bytedpypi.byted.org/simple
  Cloning https://github.com/huggingface/transformers.git (to revision main) to /tmp/pip-req-build-7v9zqvxo
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-7v9zqvxo
  Resolved https://github.com/huggingface/transformers.git to commit c8c8dffbe45ebef0a8dba4a51024e5e5e498596b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://bytedpypi.byted.org/simple
  Cloning https://github.com/huggingface/peft.git (to revision e536616888d51b453ed354a6f1e243fecb02ea08) to /tmp/pip-req-build-t02nir6n
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-t02nir6n
  Running command git rev-parse -q --verify 'sha^e536616888d51b453ed354a6f1e243fecb02ea08'
  Running command git fetch -q https://github.com/huggin

我使用了一台配置了 Python 3.10 和 Cuda 11.8 的 A100 GPU 服务器来运行本文中的代码。大约运行了一个小时。(为了验证可移植性，我还试验在Colab上运行代码，效果都很好。)

### 加载库


In [16]:
from datetime import datetime
import os
import sys

import torch
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    set_peft_model_state_dict,
)
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq



（如果出现导入​​错误，请尝试重新启动 Jupyter 内核）

### 加载数据集


In [4]:
dataset

Dataset({
    features: ['question', 'context', 'answer'],
    num_rows: 78577
})

In [17]:
from datasets import load_dataset
# dataset = load_dataset("/workspace/llm/tester-volume/llm_model/datasets", split="train")
dataset = load_dataset('json', split='train', data_files='/workspace/llm/tester-volume/llm_model/datasets/sql-create-context-v4.json')
train_dataset = dataset.train_test_split(test_size=0.1)["train"]
eval_dataset = dataset.train_test_split(test_size=0.1)["test"]
dataset


Found cached dataset json (/root/.cache/huggingface/datasets/json/default-93e9303f560821d1/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


Dataset({
    features: ['question', 'context', 'answer'],
    num_rows: 78577
})

In [19]:
print(torch.cuda.is_available())


False


In [28]:
%reset -f

上面从 Huggingface Hub 中提取数据集，并将其中的 10% 分成评估集，以检查模型在训练中的表现如何。如果您想加载自己的数据集，请执行以下操作：

```
train_dataset = load_dataset('json', data_files='train_set.jsonl', split='train')
eval_dataset = load_dataset('json', data_files='validation_set.jsonl', split='train')
```

如果您想查看数据集中的任何样本，只需执行以下操作：``` ```


In [20]:
print(train_dataset[3])

{'question': 'What is the denomination for the stamp issued 8 January 2009?', 'context': 'CREATE TABLE table_name_34 (denomination VARCHAR, date_of_issue VARCHAR)', 'answer': 'SELECT denomination FROM table_name_34 WHERE date_of_issue = "8 january 2009"'}


每个条目由文本“问题”、sql 表“上下文”和“答案”组成。

### 加载模型
我从 Huggingface 中以 int8 加载代码 llama。Lora的标准：

In [21]:
base_model = "/workspace/llm/tester-volume/llm_model/AI-ModelScope/CodeLlama-7b-hf"
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_4bit=True,
    torch_dtype=torch.float16,
    # device_map="auto",
    # low_cpu_mem_usage=True
)
tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-hf")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│    1 base_model = "/workspace/llm/tester-volume/llm_model/AI-ModelScope/CodeLlama-7b-hf"         │
│ ❱  2 model = AutoModelForCausalLM.from_pretrained(                                               │
│    3 │   base_model,                                                                             │
│    4 │   load_in_4bit=True,                                                                      │
│    5 │   torch_dtype=torch.float16,                                                              │
│                                                                                                  │
│ /workspace/llm/tester-volume/software/conda1/envs/pretrain/lib/python3.10/site-packages/transfor │
│ mers/models/auto/auto_factory.py:564 in from_pretrained                                          │
│                                                                                                  │
│   561 │   │   │   )                                                                              │
│   562 │   │   elif type(config) in cls._model_mapping.keys():                                    │
│   563 │   │   │   model_class = _get_model_class(config, cls._model_mapping)                     │
│ ❱ 564 │   │   │   return model_class.from_pretrained(                                            │
│   565 │   │   │   │   pretrained_model_name_or_path, *model_args, config=config, **hub_kwargs,   │
│   566 │   │   │   )                                                                              │
│   567 │   │   raise ValueError(                                                                  │
│                                                                                                  │
│ /workspace/llm/tester-volume/software/conda1/envs/pretrain/lib/python3.10/site-packages/transfor │
│ mers/modeling_utils.py:3669 in from_pretrained                                                   │
│                                                                                                  │
│   3666 │   │   │   hf_quantizer = None                                                           │
│   3667 │   │                                                                                     │
│   3668 │   │   if hf_quantizer is not None:                                                      │
│ ❱ 3669 │   │   │   hf_quantizer.validate_environment(                                            │
│   3670 │   │   │   │   torch_dtype=torch_dtype,                                                  │
│   3671 │   │   │   │   from_tf=from_tf,                                                          │
│   3672 │   │   │   │   from_flax=from_flax,                                                      │
│                                                                                                  │
│ /workspace/llm/tester-volume/software/conda1/envs/pretrain/lib/python3.10/site-packages/transfor │
│ mers/quantizers/quantizer_bnb_4bit.py:70 in validate_environment                                 │
│                                                                                                  │
│    67 │                                                                                          │
│    68 │   def validate_environment(self, *args, **kwargs):                                       │
│    69 │   │   if not is_accelerate_available():                                                  │
│ ❱  70 │   │   │   raise ImportError(                                                             │
│    71 │   │   │   │   f"Using `bitsandbytes` 4-bit quantization requires Accelerate: `pip inst   │
│    72 │   │   │   )                                        

torch_dtype=torch.float16 表示使用 float16 表示形式执行计算，即使值本身是 8 位整数。

如果出现错误“ValueError：Tokenizer 类 CodeLlamaTokenizer 不存在或当前未导入。”确保您的 Transformer 版本为 4.33.0.dev0 并且加速 >=0.20.3。


### 3. 检查基础型号
一个非常好的常见做法是检查模型是否已经可以完成手头的任务。微调是您要不惜一切代价尽量避免的事情：


In [ ]:
eval_prompt = """You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.
### Input:
Which Class has a Frequency MHz larger than 91.5, and a City of license of hyannis, nebraska?

### Context:
CREATE TABLE table_name_12 (class VARCHAR, frequency_mhz VARCHAR, city_of_license VARCHAR)

### Response:
"""
# {'question': 'Name the comptroller for office of prohibition', 'context': 'CREATE TABLE table_22607062_1 (comptroller VARCHAR, ticket___office VARCHAR)', 'answer': 'SELECT comptroller FROM table_22607062_1 WHERE ticket___office = "Prohibition"'}
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

输出结果：
```
SELECT * FROM table_name_12 WHERE class > 91.5 AND city_of_license = 'hyannis, nebraska'
```
如果输入只要求上课，这显然是错误的！

### 4. Tokenization
设置一些标记化设置，例如左填充，因为它使[训练使用更少的内存](https://ai.stackexchange.com/questions/41485/while-fine-tuning-a-decoder-only-llm-like-llama-on-chat-dataset-what-kind-of-pa)：

In [ ]:
tokenizer.add_eos_token = True
tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"

设置 tokenize 函数以使 labels 和 input_ids 相同。这基本上就是[自我监督微调](https://neptune.ai/blog/self-supervised-learning)：

In [ ]:
def tokenize(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=512,
        padding=False,
        return_tensors=None,
    )

    # "self-supervised learning" means the labels are also the inputs:
    result["labels"] = result["input_ids"].copy()

    return result

并运行将每个 data_point 转换为我在网上找到的效果很好的提示：

In [ ]:
def generate_and_tokenize_prompt(data_point):
    full_prompt =f"""You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.

### Input:
{data_point["question"]}

### Context:
{data_point["context"]}

### Response:
{data_point["answer"]}
"""
    return tokenize(full_prompt)

重新格式化以提示并标记每个样本：

In [ ]:
tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
tokenized_val_dataset = eval_dataset.map(generate_and_tokenize_prompt)

### 5. 设置 Lora

In [ ]:
model.train() # put model back into training mode
model = prepare_model_for_int8_training(model)

config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=[
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)

要从检查点恢复，请将resume_from_checkpoint 设置为要从中恢复的adapter_model.bin 的路径。此代码将替换连接到模型的 lora 适配器：

In [ ]:
resume_from_checkpoint = "" # set this to the adapter_model.bin file you want to resume from

if resume_from_checkpoint:
    if os.path.exists(resume_from_checkpoint):
        print(f"Restarting from {resume_from_checkpoint}")
        adapters_weights = torch.load(resume_from_checkpoint)
        set_peft_model_state_dict(model, adapters_weights)
    else:
        print(f"Checkpoint {resume_from_checkpoint} not found")

设置权重和偏差以查看训练图的可选内容：

In [ ]:
wandb_project = "sql-try2-coder"
if len(wandb_project) > 0:
    os.environ["WANDB_PROJECT"] = wandb_project


In [ ]:
if torch.cuda.device_count() > 1:
    # keeps Trainer from trying its own DataParallelism when more than 1 gpu is available
    model.is_parallelizable = True
    model.model_parallel = True

### 6. 训练参数
如果 GPU 内存不足，请更改 per_device_train_batch_size。 gradient_accumulation_steps 变量应确保这不会影响训练运行期间的批量动态。所有其他变量都是标准的东西，我不建议乱搞：

In [ ]:
batch_size = 128
per_device_train_batch_size = 32
gradient_accumulation_steps = batch_size // per_device_train_batch_size
output_dir = "sql-code-llama"

training_args = TrainingArguments(
        per_device_train_batch_size=per_device_train_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        warmup_steps=100,
        max_steps=400,
        learning_rate=3e-4,
        fp16=True,
        logging_steps=10,
        optim="adamw_torch",
        evaluation_strategy="steps", # if val_set_size > 0 else "no",
        save_strategy="steps",
        eval_steps=20,
        save_steps=20,
        output_dir=output_dir,
        # save_total_limit=3,
        load_best_model_at_end=False,
        # ddp_find_unused_parameters=False if ddp else None,
        group_by_length=True, # group sequences of roughly the same length together to speed up training
        report_to="wandb", # if use_wandb else "none",
        run_name=f"codellama-{datetime.now().strftime('%Y-%m-%d-%H-%M')}", # if use_wandb else None,
    )

trainer = Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    args=training_args,
    data_collator=DataCollatorForSeq2Seq(
        tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
    ),
)

然后我们进行一些与 pytorch 相关的优化，这只是使训练更快，但不影响准确性：

In [ ]:
model.config.use_cache = False

old_state_dict = model.state_dict
model.state_dict = (lambda self, *_, **__: get_peft_model_state_dict(self, old_state_dict())).__get__(
    model, type(model)
)
if torch.__version__ >= "2" and sys.platform != "win32":
    print("compiling the model")
    model = torch.compile(model)

In [ ]:
trainer.train()

### 加载最终检查点


In [ ]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer

base_model = "codellama/CodeLlama-7b-hf"
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-hf")

要加载经过微调的 Lora/Qlora 适配器，请使用 PeftModel.from_pretrained。 `output_dir` 应该是包含adapter_config.json和adapter_model.bin的东西：

In [ ]:
from peft import PeftModel
model = PeftModel.from_pretrained(model, output_dir)

尝试与之前相同的提示：

In [ ]:
eval_prompt = """You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.
### Input:
Which Class has a Frequency MHz larger than 91.5, and a City of license of hyannis, nebraska?

### Context:
CREATE TABLE table_name_12 (class VARCHAR, frequency_mhz VARCHAR, city_of_license VARCHAR)

### Response:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))


模型输出：
```
SELECT class FROM table_name_12 WHERE frequency_mhz > 91.5 AND city_of_license = "hyannis, nebraska"
```
从运行结果可以看到微调是有效果的！也可以将此适配器转换为 Llama.cpp 模型以在本地运行。
